In [1]:
import shap
import pandas as pd
import numpy as np

from pathlib import Path

from xgboost import XGBClassifier

In [2]:
DATA_PATH = Path('..') / 'data'
RAW_DATA_PATH = DATA_PATH / 'raw'
PROCESSED_DATA_PATH = DATA_PATH / 'processed_v3'

MODELS_PATH = Path('..') / 'models'

In [3]:
X_train = pd.read_csv(PROCESSED_DATA_PATH / 'X_train.csv')
X_val   = pd.read_csv(PROCESSED_DATA_PATH / 'X_val.csv')
y_train = pd.read_csv(PROCESSED_DATA_PATH / 'y_train.csv')
y_val   = pd.read_csv(PROCESSED_DATA_PATH / 'y_val.csv')
X_test  = pd.read_csv(PROCESSED_DATA_PATH / 'X_test.csv')
y_test  = pd.read_csv(PROCESSED_DATA_PATH / 'y_test.csv')

y_pred = np.fromfile(PROCESSED_DATA_PATH / 'y_pred.npy')
X_train

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,1,24,9,332,749,0,0,0,0,...,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00
1,0,1,24,9,306,1075,0,0,0,0,...,255,251,0.98,0.01,0.00,0.00,0.00,0.0,0.00,0.00
2,0,1,24,9,330,2655,0,0,0,0,...,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00
3,0,1,24,9,319,2452,0,0,0,0,...,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.01,0.01
4,0,2,12,9,45,113,0,0,0,0,...,255,227,0.89,0.01,0.00,0.00,0.00,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100772,0,1,24,1,0,0,0,0,0,0,...,43,247,1.00,0.00,0.02,0.19,0.00,0.0,0.95,0.99
100773,0,1,49,5,0,0,0,0,0,0,...,255,19,0.07,0.07,0.00,0.00,1.00,1.0,0.00,0.00
100774,0,1,47,9,38,130,0,0,0,0,...,49,2,0.02,0.12,0.02,1.00,0.00,0.0,0.00,0.00
100775,0,1,44,1,0,0,0,0,0,0,...,255,1,0.00,1.00,0.00,0.00,0.27,0.0,0.73,1.00


In [5]:
xgboost_model = XGBClassifier()

xgboost_model.load_model(MODELS_PATH / 'xgboost_model.json')

In [9]:
X_train_for_shap = X_train.iloc[:22500,:]
X_test_for_shap = X_test.iloc[:22500,:]

In [10]:
explainer_train = shap.TreeExplainer(xgboost_model, X_train_for_shap)
explainer_test = shap.TreeExplainer(xgboost_model, X_test_for_shap)

In [11]:
shap_values_train = explainer_train.shap_values(X_train_for_shap)
shap_values_test = explainer_test.shap_values(X_test_for_shap)

 99%|===================| 22296/22500 [01:16<00:00]        

In [9]:
#shap.plots.bar(shap_values_train)
#shap.summary_plot(shap_values_train, X_train_for_shap, plot_type="bar")
# shap_values_test.shape

(10000, 41)

In [12]:
shap_values_train.tofile(PROCESSED_DATA_PATH / 'shap_values_train_from_xgboost.npy')
shap_values_test.tofile(PROCESSED_DATA_PATH / 'shap_values_test_from_xgboost.npy')